<a href="https://colab.research.google.com/github/valeriarpo/Business-Analytics/blob/main/4_RETO2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Valeria Restrepo Betancur
Reto 2 - Arboles de decisión
0.Cargar las librerias de decisión

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Se cargan los datos desde el drive

In [ ]:
nxl = "/content/drive/MyDrive/ANALITICA DE NEGOCIOS/2. BD2_In_App_Pur_Int.xlsx"
XDB = pd.read_excel(nxl, sheet_name= 0)
XDB= XDB.dropna()    #Evitar celdas vacias
XDB.head()
#Seleccionamos las variables de interés
XD=XDB[["Age", "Income", "Years", "Hours"]]
XD.head()                                         #Se necesita array para que los lea como números
XD=np.array(XD)           #Convierte la tabla en números
XD
yd=XDB[["Buy"]] ; yd=np.array(yd)       #Variable de Salida - Convierte a la tabla



2. Se procede con la construcción del modelo de árbol

In [ ]:
from re import S
mar = DecisionTreeClassifier(criterion="gini", max_depth= 4)
mar.fit(XD,yd)     #Valores de pronóstico sirven para evaluar el modelo
ydp = mar.predict(XD)

#Evaluamos el modelo
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(yd, ydp)
VN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
VP = cm[1][1]
print(cm)
Exactitud = (VP+VN)/(VP+VN+FP+FN)
Exactitud
Sensibilidad = VP/(VP+FN)
Sensibilidad
Especificidad = VN/(VN+FP)
Especificidad

print("Exactitud", Exactitud, "Sensibilidad", Sensibilidad, "Especificidad", Especificidad)


[[139  50]
 [ 11 200]]
Exactitud 0.8475 Sensibilidad 0.9478672985781991 Especificidad 0.7354497354497355


3. Se procede con la visualización del árbol

In [ ]:
from sklearn.tree import export_graphviz      #export graphiz es graficador
from pydotplus import graph_from_dot_data
from IPython.display import Image
from google.colab import files
vs= ["Age", "Income", "Years", "Hours"] #Estos nombres aparecen en el arbol

dot_data=export_graphviz(mar, feature_names=vs)
graph= graph_from_dot_data(dot_data)      #Se hace gráfico del Árbol
graph.write_png("arbol.png")            #Se crea la imagén
Image("arbol.png")
files.download('arbol.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

4. Le calculamos los valores de compra a los solicitantes nuevos

In [ ]:
XDB2=pd.read_excel(nxl, sheet_name=1)   #Hoja dos es la que no tiene salida
XDB2=XDB2[["Age", "Income", "Years", "Hours"]]
XDB2=XDB2.dropna()   #Base de datos solicitantes nuevos

XD2=np.array(XDB2)
ydp2=mar.predict(XD2)
XDB2["Buy"]=ydp2     #Le pego la columna a la base de datos
XDB2.head()

XDB2.to_excel("NuevosSolicitantes1.xlsx")   #Importancia del analísis
files.download('NuevosSolicitantes1.xlsx')

len(ydp2)
compradores = np.sum(ydp2==1)
print("El número de compradores es", compradores)
no_comprador = len(ydp2)-np.sum(ydp2==1)
print("El número de no compradores es", no_comprador)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

El número de compradores es 1
El número de no compradores es 1


**Analísis de resultados**
En la base de datos se puede observar un total de 400 individuos alrededor de 189 no compraron en la aplicación (47.25%), y un total de 211 individuos compraron en la app (52.75%). De acuerdo con lo anterior, el modelo de árbol mostró una exactitud de 84.75%, lo cual es positivo pues muestra la alta precisión en analizar todos los datos, adicionalmente el indice de sensibilidad es del 94.7%, esto muestra la alta exactitud del modelo al analizar a los compradores. Por otro lado la especificidad fue del 73.5%, esta es relativamente buena al analizar los no compradores.
En los dos nuevos jugadores, el primero tiene una probabilidad de comprar dentro de la aplicación de 66.9% y de no comprar de 33%, lo cual nos da a entender que hay mayor probabilidad de que compre el primer jugador. El segundo jugador tiene una probabilidad del 100% de no comprar en la aplicación.

El primer nodo puro es de un ingreso menor o igual a 63.5, lo cual nos da una probabilidad del 100% de que los jugadores que tengan un ingreso menor no compren en la app. El segundo nodo puro tiene como regla tener mas years que 0.5. Y como ultimo, el tercer nudo puro es con una edad mayor a 23 años de edad.
Las reglas para los nodos Puro:
1. Income <= 63.5
2. Income >63.5 AND Years <=1.5 AND Years > 0.5
3. Income >63.5 AND Years <=1.5 AND Years <= 0.5 AND Age > 23

Las reglas para los nuevos casos son las siguientes:
1. Income >63.5 AND Years >1.5 AND Years <= 5.5 AND Income > 76.5
2.  Income >63.5 AND Years <=1.5 AND Years > 0.5 (28,0)

